## Gathering Data for Analysis
This project will be based on data from the following sources:
a) Foursquare, which will provide location data for the southeastern Manitoba region and amenities located within it, and b) Trail Forks, which identifies the most established mountain biking trails in a region

This project will take the geographic locations of popular cycling routes from Trail Forks as one dataset. It will take the geographic locations of trails from Trail Forks as another dataset. It will plot both on a map and show their location relative to area hotels. 

The results will provide the event organizer with a set of possible venues for a mountain bike race in southeastern Manitoba. From there, the organizer can consider other factors, such as proximity of restaurants that could potentially cater a post-race meal, or features of the possible trails (such as length of the trail or nature of the terrain) that could further inform the decision-making process to select an event venue.

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [3]:
CLIENT_ID = 'KOYZEXK0ES5ID3C1WLGKIHXJBWRIS4MYQJU2SKEFS333X5DQ' # your Foursquare ID
CLIENT_SECRET = 'KHRSIOGE3FCDQ1IN2BSPIBKKAO1EH2XU3G1LIKWO3JTPG0X0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KOYZEXK0ES5ID3C1WLGKIHXJBWRIS4MYQJU2SKEFS333X5DQ
CLIENT_SECRET:KHRSIOGE3FCDQ1IN2BSPIBKKAO1EH2XU3G1LIKWO3JTPG0X0


In [4]:
address = 'Steinbach, Manitoba, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

49.530557 -96.69342


In [5]:
search_query='hotel'
radius=200000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KOYZEXK0ES5ID3C1WLGKIHXJBWRIS4MYQJU2SKEFS333X5DQ&client_secret=KHRSIOGE3FCDQ1IN2BSPIBKKAO1EH2XU3G1LIKWO3JTPG0X0&ll=49.530557,-96.69342&v=20180604&query=hotel&radius=200000&limit=30'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef4f89d353f133543654fee'},
 'response': {'venues': [{'id': '4f6c709fbb3d8e65c21090e4',
    'name': 'La Broquerie Hotel',
    'location': {'address': '3 Pignons St',
     'lat': 49.517595,
     'lng': -96.502317,
     'labeledLatLngs': [{'label': 'display',
       'lat': 49.517595,
       'lng': -96.502317}],
     'distance': 13884,
     'postalCode': 'R0A 0W0',
     'cc': 'CA',
     'city': 'La Broquerie',
     'state': 'MB',
     'country': 'Canada',
     'formattedAddress': ['3 Pignons St',
      'La Broquerie MB R0A 0W0',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1593113173',
    'hasPerk': False},
   {'id': '4e2361e6e4cdf68591996eb5',
    'name': 'St Anne Hotel Chinese Restaurant',
   

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4f6c709fbb3d8e65c21090e4,La Broquerie Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1593113173,False,3 Pignons St,49.517595,-96.502317,"[{'label': 'display', 'lat': 49.517595, 'lng':...",13884,R0A 0W0,CA,La Broquerie,MB,Canada,"[3 Pignons St, La Broquerie MB R0A 0W0, Canada]",NaN,NaN,NaN
1,4e2361e6e4cdf68591996eb5,St Anne Hotel Chinese Restaurant,[],v-1593113173,False,NaN,49.672748,-96.657028,"[{'label': 'display', 'lat': 49.67274811812094...",16044,NaN,CA,St. Anne,MB,Canada,"[St. Anne MB, Canada]",NaN,NaN,NaN
2,4d8239892fb73704a4238811,The Alouette Hotel,[],v-1593113173,False,101 2nd St. E,49.673719,-96.822029,"[{'label': 'display', 'lat': 49.673719, 'lng':...",18440,R0A 0X0,CA,The Pas,MB,Canada,"[101 2nd St. E, The Pas MB R0A 0X0, Canada]",NaN,NaN,NaN
3,50fb1e1ee4b045129eff1d63,St. Malo Hotel,"[{'id': '4bf58dd8d48988d118941735', 'name': 'D...",v-1593113173,False,NaN,49.315830,-96.954730,"[{'label': 'display', 'lat': 49.31583, 'lng': ...",30485,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN,NaN
4,56254264498ebd5d2e1b8916,Ste Anne Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1593113173,False,103 Dawson Rd,49.657334,-96.455816,"[{'label': 'display', 'lat': 49.6573337, 'lng'...",22206,R5H 1B6,CA,Ste. Anne,MB,Canada,"[103 Dawson Rd, Ste. Anne MB R5H 1B6, Canada]",NaN,NaN,NaN


In [11]:
data = {'Place_name':['Sandilands', 'St. Malo', 'Granite Groove Out', 'McGillivray Falls', 'Falcon Lake Trans Canada Trail', 'Falcon Ridge Ski Slopes', 'Crystal Springs Colony'],
        'Latitude': [49.3246, 49.3141, 50.1503, 49.8095, 49.7030, 49.690300,  49.546950],
        'Longitude': [-96.2932, -96.9536, -95.8798, -95.2379, -95.2433, -95.317180, -97.116850],
        'Green_circle': [8, 4, 3, 3, 13, 22, 2],
        'Blue_square': [12, 0, 4, 18, 0, 21, 4],
        'Black_diamond': [1, 0, 0, 0, 0, 2, 0],
        'Total_distance': [16, 5.2, 7.2, 31, 28, 32, 2.5]}

dfTrailForks = pd.DataFrame(data,columns=['Place_Name', 'Latitude', 'Longitude', 'Green_circle', 'Blue_square', 'Black_diamond', 'Total_distance'])

#Express the values in Place_Name as a string
data['Place_name'] = str('Place_name')

In [12]:
dfTrailForks.head()

,Place_Name,Latitude,Longitude,Green_circle,Blue_square,Black_diamond,Total_distance
0,NaN,49.3246,-96.2932,8,12,1,16.0
1,NaN,49.3141,-96.9536,4,0,0,5.2
2,NaN,50.1503,-95.8798,3,4,0,7.2
3,NaN,49.8095,-95.2379,3,18,0,31.0
4,NaN,49.7030,-95.2433,13,0,0,28.0
